In [1]:
import string
import sys
import pandas as pd
from pandas import DataFrame
import os
import natsort
import matplotlib.pyplot as plt

## 데이터 취합 & 분석

### 001.데이터 작업할 CSV파일 list로 만들기

In [39]:
### NACL 정보 로드 
import os

folder_path = './Desalination_Analysis_NACL/' # 각 샘플 정보를 다 읽음
csv_file_list = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv') and file_name.startswith('MD'):
        csv_file_list.append(file_name)
        
csv_file_listNACL = csv_file_list

In [41]:
### WATER 정보 로드 
import os

folder_path = './Desalination_Analysis_WATER_COUNT/' # 각 샘플 정보를 다 읽음
csv_file_list = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv') and file_name.startswith('MD'):
        csv_file_list.append(file_name)
        
csv_file_listWATER = csv_file_list

### 002. NACL count정보와  WATER count 정보, 두 CSV에 있는 데이터를 각 범위에 따라서 feed 농도를 설정하고 Permeate 농도를 우선 구하고 두 농도를 나눈값 (Rejection)

$R=$ $\left(1-\frac{C_Permeate}{C_Feed} \right) \times 100$

In [35]:
mean_values = pd.DataFrame()  # 빈 데이터프레임으로 초기화

for i in csv_file_listWATER:
    DATA_WATER = pd.read_csv("./Desalination_Analysis_WATER_COUNT/"+str(i))
    DATA_Salt = pd.read_csv("./Desalination_Analysis_NACL/"+str(i))
    ###################################################################################
    selected_columns = [col for col in DATA_Salt.columns if 'NA' in col]
    DATA_NA = DATA_Salt[selected_columns]
    new_columns = [col.replace("NA_", "") if "NA_" in col else col for col in DATA_NA.columns]
    DATA_NA.columns  = new_columns
    ###################################################################################
    selected_columns = [col for col in DATA_Salt.columns if 'CL' in col]
    DATA_CL = DATA_Salt[selected_columns]
    new_columns = [col.replace("CL_", "") if "CL_" in col else col for col in DATA_CL.columns]
    DATA_CL.columns  = new_columns
    ###################################################################################
    if '0.65M' in i:
        DATA = (1 - ((DATA_CL)/(DATA_WATER)) / ((117)/(9000))) * 100  
    elif '1.7M' in i:
        DATA= (1 - ((DATA_CL)/(DATA_WATER)) / ((306)/(9000))) * 100 
    elif '3.4M' in i:
        DATA = (1 - ((DATA_CL)/(DATA_WATER)) / ((612)/(9000))) * 100 
    ###################################################################################
    mean_values = pd.concat([mean_values, DATA[19:].mean()], axis=1,sort=False)
    
modified_list = [csv[3:-4] for csv in csv_file_list]    
mean_values.columns = modified_list
mean_values.to_csv("SUMMARY_NACL_Rejection_Percent_Ver2.csv", mode='a')
